## Etat initial

In [2]:
import numpy as np

# Initialiser un tenseur 8x8x12 avec des zéros
initial_state = np.zeros((8, 8, 12), dtype=int)
insample = np.zeros((1,8,8,12), dtype=int)

# Pions blancs (1er étage)
initial_state[:, :, 0] = np.array([
    [0, 0, 0, 0, 0, 0, 0, 0],
    [1, 1, 1, 1, 1, 1, 1, 1],
    [0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0]
]).T

# On transpose pour faciliter l'indexation.

# Pions noirs (2e étage)
initial_state[:, :, 1] = np.array([
    [0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0],
    [1, 1, 1, 1, 1, 1, 1, 1],
    [0, 0, 0, 0, 0, 0, 0, 0]
]).T

# Roi blanc (3e étage)
initial_state[4, 0, 2] = 1

# Roi noir (4e étage)
initial_state[4, 7, 3] = 1

# Reine blanche (5e étage)
initial_state[3, 0, 4] = 1

# Reine noire (6e étage)
initial_state[3, 7, 5] = 1

# Tours blanches (7e étage)
initial_state[0, 0, 6] = 1
initial_state[7, 0, 6] = 1

# Tours noires (8e étage)
initial_state[0, 7, 7] = 1
initial_state[7, 7, 7] = 1

# Fous blancs (9e étage)
initial_state[2, 0, 8] = 1
initial_state[5, 0, 8] = 1

# Fous noirs (10e étage)
initial_state[2, 7, 9] = 1
initial_state[5, 7, 9] = 1

# Cavaliers blancs (11e étage)
initial_state[1, 0, 10] = 1
initial_state[6, 0, 10] = 1

# Cavaliers noirs (12e étage)
initial_state[1, 7, 11] = 1
initial_state[6, 7, 11] = 1

insample[0] = initial_state

In [3]:
etage = {"p": 0, "P": 1,
         "k": 2, "K": 3,
         "q": 4, "Q": 5,
         "r": 6, "R": 7,
         "b": 8, "B": 9,
         "n":10, "N":11 }

## Encodage du coup `1 - e4`

In [4]:
# e2 de coordonnées (4, 1)
# e4 de coordonnées (4, 3)

outpos_index = lambda i, j : 8*i + j

e4 = np.zeros((64,64), dtype=int)
e4[outpos_index(4,1), outpos_index(4,3)] = 1

outsample = np.zeros((1,64,64), dtype=int)
outsample[0] = e4

## Adaptation structure de données

### Conversion d'une board

In [5]:
import chess.pgn
import pandas as pd
import numpy as np

In [6]:
with open("aggressive_games_1400.pgn") as pgn_file:
    game0 = chess.pgn.read_game(pgn_file)
game0.board().__repr__()

"Board('rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1')"

In [7]:
def pgn_pos_conv(game):
    board = np.zeros((8, 8, 16), dtype=int)
    pgn0 = game.board().epd()
    splits = pgn0.split(" ")
    pieces, additional = splits[0], splits[1:]
    rows = pieces.split("/")
    i=0
    for row in rows:
        j = 0
        for piece in row:
            if piece.isdigit():
                j += int(piece)
            else:
                board[i, j, etage[piece]] = 1
                j += 1
        i += 1
    # 13e étage - Trait au blanc (1) ou noirs (0)
    tour = additional[0]
    board[:, :, 12] = np.ones((8,8)) * int(tour == "w")

    # 14e étage - Possibilité de roquer
    roques = additional[1]
    board[0:2, :, 13] = np.ones((2,8)) * int("K" in roques)
    board[2:4, :, 13] = np.ones((2,8)) * int("Q" in roques)
    board[4:6, :, 13] = np.ones((2,8)) * int("k" in roques)
    board[6:8, :, 13] = np.ones((2,8)) * int("q" in roques)

    # 15e étage - Possibilité de pep pour les blancs
    pep = additional[2]
    if "f" in pep :
        row = int(pep[1])
        board[row, :, 14] = np.ones(8)
    
    # 16e étage - Possibilité de pep pour les noirs
    if "c" in pep :
        row = int(pep[1])
        board[row, :, 15] = np.ones(8)

    return board

In [8]:
state = pgn_pos_conv(game0)

### Conversion d'un move

In [9]:
outpos_index = lambda i, j : 8*i + j

def move_encoding(next_node):
    res = np.zeros((64, 64))
    x = {'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 4, 'f': 5, 'g': 6, 'h': 7}
    y = {str(i+1) : i for i in range(8)}
    
    xd, yd, xa, ya = list(str(next_node.move))[:4]
    # print(next_node.move)
    # print(x[xd], y[yd], x[xa], y[ya])
    res[outpos_index( x[xd] , y[yd] ), outpos_index( x[xa] , y[ya] )] = 1
    return res

In [10]:
move_encoding(game0.variation(0))[outpos_index(4,1), outpos_index(4,3)]

np.float64(1.0)

Cohérent (la partie commence par `1 - e4`)

__Nombre de moves__

In [11]:
Nmoves = 0

node = game0
while node.variations:
    next_node = node.variation(0)
    Nmoves += 1
    node = next_node

In [12]:
def convert_game(game):
    Nmoves = 0

    node = game
    while node.variations:
        next_node = node.variation(0)
        Nmoves += 1
        node = next_node

    states = np.zeros((Nmoves, 8, 8, 16))
    moves = np.zeros((Nmoves, 64, 64))

    node = game
    i = 0
    while node.variations:
        next_node = node.variation(0)
        states[i, :, :, :] = pgn_pos_conv(node)
        moves[i, :, :] = move_encoding(next_node)
        i += 1
        node = next_node
    return states, moves

In [13]:
states, moves = convert_game(game0)

## Modèle de réseau de neurones

In [21]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense, Reshape

In [25]:
model = Sequential([
    Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(8, 8, 16)),
    Conv2D(64, kernel_size=(3, 3), activation='relu'),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(64 * 64, activation='softmax'),
    Reshape((64, 64))
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Afficher un résumé du modèle
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_4 (Conv2D)               │ (None, 6, 6, 32)       │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 4, 4, 64)       │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 128)            │       131,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 4096)           │       528,384 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape_2 (Reshape)             │ (None, 64, 64)         │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 682,720 (2.60 MB)

 Trainable params: 682,720 (2.60 MB)

 Non-trainable params: 0 (0.00 B)

## Entrainement du modèle (sans pré-clustering)

### Nombre de games

In [20]:
N_games = 0
with open("aggressive_games_1400.pgn") as pgn_file:
    game = chess.pgn.read_game(pgn_file)
    while game :
        N_games += 1
        game = chess.pgn.read_game(pgn_file)
N_games

8062

In [26]:
N_test = 8062//5

with open("aggressive_games_1400.pgn") as pgn_file:
    
    # On skippe les games de test.
    for _ in range(N_test) :
        chess.pgn.read_game(pgn_file)

    n_game=1
    game = chess.pgn.read_game(pgn_file)
    
    # Puis on s'entraine sur le sample.
    while game :
        states, moves = convert_game(game)
        model.fit(states, moves, epochs=1, batch_size=1)
        game = chess.pgn.read_game(pgn_file)
        if n_game%10==0 : print(f"---------------------------------- {n_game} ----------------------------------")
        n_game += 1
    
model_fname = "agressive_bot.h5"
model.save(model_fname)

### Sauvegarde du modèle

In [ ]:
model_fname = "agressive_bot.h5"
model.save(model_fname)

---

Pour importer le modèle :

```python
from tensorflow.keras.models import load_model

# Load the entire model
loaded_model = load_model('my_model.h5')
```
---

In [147]:
rev_outpos_index = lambda n : (n//8, n%8)

prediction = model.predict(states[0:, :, :, :])[0]
argmax = prediction.argmax()

dep, arr = np.unravel_index(argmax, prediction.shape)
rev_outpos_index(dep), rev_outpos_index(arr)

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 180ms/step


((np.int64(1), np.int64(7)), (np.int64(2), np.int64(5)))